In [1]:
#!/usr/bin/python

import numpy as np
import math
from astropy.io import fits as fits

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

import pandas as pd
from pandas import DataFrame, Series

from astropy.wcs import WCS as WCS

import import_ipynb
import Pipe3D_Asier_Test as Pipe3D

from mpdaf.obj import hms2deg
from mpdaf.obj import dms2deg

from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

from scipy.stats import ks_2samp
from scipy import stats

importing Jupyter notebook from Pipe3D_Asier_Test.ipynb


In [2]:
alfa = 1.0
delta = 500
sigma = 0.1

In [ ]:
#Crear arrays con los datos de todas las galaxias.

Numb_Lines = len(open('Names_Amusing_true.txt').readlines())

All_SN_type = np.zeros(Numb_Lines)
All_Prob_II = np.zeros(Numb_Lines)
All_Cumul_II = np.zeros(Numb_Lines)
All_Prob_Ia = np.zeros(Numb_Lines)
All_Cumul_Ia = np.zeros(Numb_Lines)

In [ ]:
names = open('Names_Amusing_true.txt')

Galaxy = -1
nIa = 0
nII = 0
for lines in names:
    line= lines[:-1]
    print(line)
    Galaxy = Galaxy+1

    df = pd.read_csv('Amusing_Table_true.csv')  # To find the Galaxy data
    Names_1 = df['SN_name']
    Names_2 = df['def_name']
    R = 0

    for names in Names_1:              # Finding the Galaxy in the DataFrame
        R=R+1
        if line == names:
            Row = R-1
    R = 0
    for names in Names_2:              # Some Galaxies have the name in other column
        R=R+1
        if line == names:
            Row = R-1
    
    # SN coordinates
    RA_SN = df.at[Row,'RA_SN']
    RA_SN = hms2deg(RA_SN)
    
    DEC_SN = df.at[Row,'DEC_SN']
    DEC_SN = dms2deg(DEC_SN)
    
    # Galaxy center coordinates
    RA_GAL = df.at[Row,'RA_GAL']
    RA_GAL = hms2deg(RA_GAL)
    
    DEC_GAL = df.at[Row,'DEC_GAL']
    DEC_GAL = dms2deg(DEC_GAL)
    
    # Pixel position
    w = WCS('../Data/Amusing/flux_elines.'+line+'.cube.fits')
    Pix_X, Pix_Y ,z = w.wcs_world2pix(RA_SN, DEC_SN, 0. , 1)
    Pix_X_GAL, Pix_Y_GAL ,z = w.wcs_world2pix(RA_GAL, DEC_GAL, 0. , 1)
    
    # Integer for measure
    Pix_X = int(Pix_X)
    Pix_Y = int(Pix_Y)
    
    # Supernova type
    if df.at[Row,'SN_type'] == 'Ia':
        SN_TYPE = 1
        nIa = nIa + 1
    elif df.at[Row,'SN_type'] == 'II':
        SN_TYPE = 2
        nII = nII + 1
    else:
        SN_TYPE = 0
    
#~~~~~~~~~~~~ SN Rates ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # Call to Pipe3D code
    line = Pipe3D.Pipe3D_galaxy(line)
    
    # Obtain the maps of the galaxy
    line.painting_galaxy_maps(False, alfa, delta, sigma)
    
    # Gaussian function
    stddev = 7
    gauss = Gaussian2DKernel(stddev)
    
    # Gaussian convolve
    line.mass_density_map = convolve(line.mass_density_map, gauss)
    line.SNII_rate_map = convolve(line.SNII_rate_map, gauss)
    line.SNIa_rate_map = convolve(line.SNIa_rate_map, gauss)
    
    # Eliminate negative values
    line.SNII_rate_map[line.SNII_rate_map<0.0] = 0.0
    line.SNIa_rate_map[line.SNIa_rate_map<0.0] = 0.0
    
# ------------- SN II ------------------------------------------------------------------------------------
    
    # An Array whit all the value of the SN II rate map
    line.Prob_List_II = np.ndarray.flatten(line.SNII_rate_map)
    # The array orther in value
    line.Order_Prob_List_II = np.sort(line.Prob_List_II)
    
    # Normalize
    line.Norm_SNII = 0
    for i in range(len(line.Order_Prob_List_II)):
        line.Norm_SNII = line.Norm_SNII + line.Order_Prob_List_II[i]

    line.Order_Prob_List_II = line.Order_Prob_List_II / line.Norm_SNII
    line.SNII_rate_map = line.SNII_rate_map / line.Norm_SNII
    
    # Calculate P
    line.cumulative_P_values_II = np.cumsum(line.Order_Prob_List_II)

    # Calculate P map
    line.cumulative_SNII_rate_map = np.interp(line.SNII_rate_map, line.Order_Prob_List_II, line.cumulative_P_values_II)
    
    # Measure Probability
    line.Prob_Val_SNII = 0
    line.Cumulative_Prob_SNII = 0
    square = np.arange(3)-1
    
    for j in square:
        for i in square:
            # The probability of SNII (p)
            line.Prob_Val_SNII = line.Prob_Val_SNII + line.SNII_rate_map[Pix_Y+j,Pix_X+i]
            # The cumulative probability of SNII (P)
            line.Cumulative_Prob_SNII = line.Cumulative_Prob_SNII + line.cumulative_SNII_rate_map[Pix_Y+j,Pix_X+i]
    
    line.Prob_Val_SNII = line.Prob_Val_SNII / 9
    line.Cumulative_Prob_SNII = line.Cumulative_Prob_SNII/9
    
# ------------- SN Ia ------------------------------------------------------------------------------------
    
    # An Array whit all the value of the SN Ia rate map
    line.Prob_List_Ia = np.ndarray.flatten(line.SNIa_rate_map)
    # The array orther in value
    line.Order_Prob_List_Ia = np.sort(line.Prob_List_Ia)
    
    # Normalize
    line.Norm_SNIa = 0
    for i in range(len(line.Order_Prob_List_Ia)):
        line.Norm_SNIa = line.Norm_SNIa + line.Order_Prob_List_Ia[i]

    line.Order_Prob_List_Ia = line.Order_Prob_List_Ia / line.Norm_SNIa
    line.SNIa_rate_map = line.SNIa_rate_map / line.Norm_SNIa
    
    # Calculate P
    line.cumulative_P_values_Ia = np.cumsum(line.Order_Prob_List_Ia)

    # Calculate P map
    line.cumulative_SNIa_rate_map = np.interp(line.SNIa_rate_map, line.Order_Prob_List_Ia, line.cumulative_P_values_Ia)
    
    # Measure Probability
    line.Prob_Val_SNIa = 0
    line.Cumulative_Prob_SNIa = 0
    square = np.arange(3)-1
    
    for j in square:
        for i in square:
            # The probability of SNIa (p)
            line.Prob_Val_SNIa = line.Prob_Val_SNIa + line.SNIa_rate_map[Pix_Y+j,Pix_X+i]
            # The cumulative probability of SNIa (P)
            line.Cumulative_Prob_SNIa = line.Cumulative_Prob_SNIa + line.cumulative_SNIa_rate_map[Pix_Y+j,Pix_X+i]
    
    line.Prob_Val_SNIa = line.Prob_Val_SNIa / 9
    line.Cumulative_Prob_SNIa = line.Cumulative_Prob_SNIa/9
    
# ------------- Save Data ------------------------------------------------------------------------------------
    
    # SN type
    All_SN_type[Galaxy] = SN_TYPE
    # SN II and cumulative SN II Probability
    All_Prob_II[Galaxy] = line.Prob_Val_SNII
    All_Cumul_II[Galaxy] = line.Cumulative_Prob_SNII
    # SN Ia and cumulative SN Ia Probability
    All_Prob_Ia[Galaxy] = line.Prob_Val_SNIa
    All_Cumul_Ia[Galaxy] = line.Cumulative_Prob_SNIa
    
    
    
    
    data_CC = np.column_stack((All_SN_type,All_Prob_II,All_Cumul_II))
    np.savetxt('../Results_CC_'+str(sigma)+'.txt',data_CC)
    
    data_Ia = np.column_stack((All_SN_type,All_Prob_Ia,All_Cumul_Ia))
    np.savetxt('../Results_Ia_'+str(alfa)+'_'+str(delta)+'.txt',data_Ia)
    
    # Mass density map
    plt.imshow(line.mass_density_map)
    plt.colorbar()
    plt.plot([Pix_X_GAL],[Pix_Y_GAL],'k+')
    plt.plot([Pix_X],[Pix_Y],'kx')
    plt.title('{}: Mass Density map (Mo/parsec**2)' .format(line.name))
    plt.savefig('../result/'+line.name+'_mass_density_map.png')
    #plt.show()
    plt.close()

    # Log Mass density map
    plt.imshow(line.mass_density_map, norm=LogNorm())
    plt.colorbar()
    plt.plot([Pix_X_GAL],[Pix_Y_GAL],'k+')
    plt.plot([Pix_X],[Pix_Y],'kx')
    plt.title('{}: Log Mass Density map' .format(line.name))
    plt.savefig('../result/'+line.name+'_mass_density_map_Log.png')
    #plt.show()
    plt.close()
    
        # Log SN II rate map
    plt.imshow(line.SNII_rate_map, norm=LogNorm(), vmin=1e-8, vmax=1e-4)
    plt.colorbar()
    plt.plot([Pix_X_GAL],[Pix_Y_GAL],'k+')
    plt.plot([Pix_X],[Pix_Y],'kx')
    plt.title('{}: Log star formation rate map' .format(line.name))
    plt.savefig('../result/'+line.name+'_SNII_map_Log.png')
    #plt.show()
    plt.close()
    
        # SN II P map
    plt.imshow(line.cumulative_SNII_rate_map, vmin=0, vmax=1)
    plt.colorbar()
    plt.plot([Pix_X_GAL],[Pix_Y_GAL],'k+')
    plt.plot([Pix_X],[Pix_Y],'kx')
    plt.title('{}: star formation rate P map' .format(line.name))
    plt.savefig('../result/'+line.name+'_P_Map_II.png')
    #plt.show()
    plt.close()
    
        # Log SN Ia Rate Map
    plt.imshow(line.SNIa_rate_map, norm=LogNorm(), vmin=1e-8, vmax=1e-4)
    plt.colorbar()
    plt.plot([Pix_X_GAL],[Pix_Y_GAL],'k+')
    plt.plot([Pix_X],[Pix_Y],'kx')
    plt.title('{}: Log SN Ia rate map' .format(line.name))
    plt.savefig('../result/'+line.name+'_SNIa_map_Log.png')
    #plt.show()
    plt.close()
    
        # SN Ia P map
    plt.imshow(line.cumulative_SNIa_rate_map, vmin=0, vmax=1)
    plt.colorbar()
    plt.plot([Pix_X_GAL],[Pix_Y_GAL],'k+')
    plt.plot([Pix_X],[Pix_Y],'kx')
    plt.title('{}: SN Ia rate P map' .format(line.name))
    plt.savefig('../result/'+line.name+'_P_Map_Ia.png')
    #plt.show()
    plt.close()

ASASSN13an
ASASSN13ch
ASASSN13cj
ASASSN13cp
ASASSN13dm
ASASSN14at
ASASSN14ba
ASASSN14bf
ASASSN14co
ASASSN14cu
ASASSN14db
ASASSN14dz
ASASSN14hr
ASASSN14ig
ASASSN14ko
ASASSN14lt
ASASSN14lu


KeyboardInterrupt: 